In [2]:
import pickle
import pandas as pd

In [3]:
data_dir = '/lustre/groups/epigenereg01/workspace/projects/vale/MLM/agarwal_2022/'

In [16]:
folds_df = pd.read_csv(data_dir + 'data/saluki_paper/Fig3_S4/binnedgenes.txt', sep='\t', usecols=[0,1],
                      names=['Fold','gene_id'], skiprows=1)#folds as they are in Agarwal article

In [17]:
transcript_to_gene = pd.read_csv(data_dir + '../UTR_coords/GRCh38_EnsembleCanonical_HGNC.tsv.gz', sep='\t', 
                                     names=['gene_id','transcript_id'], skiprows=1,usecols=[0,1])

In [34]:
used_transcripts = transcript_to_gene[transcript_to_gene.gene_id.isin(folds_df.gene_id)].transcript_id.values

In [24]:
input_fasta = data_dir + '../fasta/Homo_sapiens_no_reverse.fa'
output_fasta = data_dir + '/fasta/Homo_sapiens_dna_fwd.fa'

In [49]:
faidx = pd.read_csv(input_fasta + '.fai', sep='\t',header=None,names=['name','length','offset','linebases','linewidth'])
faidx['transcript_id'] = faidx.name.apply(lambda x:x.split('.')[0])
faidx = faidx[faidx.transcript_id.isin(used_transcripts)].drop(columns='transcript_id')

faidx.name.to_csv(data_dir + '/fasta/regions.tsv',index=None,header=None)

In [50]:
! samtools faidx -r {data_dir + '/fasta/regions.tsv'} {input_fasta} > {output_fasta}

In [51]:
! samtools faidx {output_fasta}